## Launch an Inference Server (vLLM) for the Compressed Model

This step sets up a vLLM inference server to host your compressed model and exposes an OpenAI-compatible API endpoint. This server is required so that GuideLLM can benchmark system-level performance like throughput, latency, and time-to-first-token.

**Goal**: Make the compressed model accessible via an API for performance evaluation.

**Output**: vLLM server running with the compressed model, ready to accept requests.


In [44]:
import os
import time
import torch
from vllm import LLM, SamplingParams
from utils import generate

In [45]:
# set the logging level for vLLM inference
os.environ["VLLM_LOGGING_LEVEL"] = "DEBUG"

### vLLM config for single node
For **single-node**, **single-GPU** or **multi-GPU (but not multinode)** vLLM serving, the main arguments are:

``--model``: Model path or Hugging Face repo ID (required).

``--tensor-parallel-size``: Number of GPUs to use (set to 1 for single GPU, or >1 for multi-GPU tensor parallelism).

``--port``: Port for the API server (default is 8000).

``--host``: Host IP address (default is 127.0.0.1).

``--gpu-memory-utilization``: controls what fraction of each GPU’s memory vLLM will use for the model executor and KV cache. For example, --gpu-memory-utilization 0.5 means vLLM will use 50% of the GPU memory.

``--quantization``: Method used to quantize the weights. 
 
``--max-model-len``: argument sets the maximum context length (in tokens) that vLLM can process for both prompt and output combined. If not specified, it defaults to the model’s config value. Setting a higher value allows longer prompts/completions but increases GPU memory usage for the KV cache; setting it lower saves memory but limits context length. Set this to prevent problems with memory if the model’s default context length is too long.

For **multi-node** vLLM serving, use:

 ``--tensor-parallel-size`` Number of GPUs per node (or total GPUs if not using pipeline parallelism).
 
``--pipeline-parallel-size`` Number of nodes (optional, for pipeline parallelism).

Additionally, for multi-node setup, a Ray cluster is also needed.



# Run this command in terminal to serve the compressed model using vLLM
```
vllm serve \
  --model "compressed_model" \
  --host 127.0.0.1 \
  --port 8000 \
  --gpu-memory-utilization 0.8 \
  --tensor-parallel-size 1 \
  --pipeline-parallel-size 1
```

Once the server starts, you will see something like this:

```INFO:     Started server process [37883]```\
```INFO:     Waiting for application startup.```\
```INFO:     Application startup complete.```


## A test run to see if the vLLM server is accessible
We use a helper function **generate** (defined in [utils.py](./utils.py)) to simplify sending requests to our locally-served VLLM model.
This function wraps the OpenAI-compatible Chat Completions API exposed by VLLM.
### Why we Use the OpenAI SDK with vLLM
vLLM implements an OpenAI-compatible REST API, meaning:

- vLLM starts a local web server (e.g., http://127.0.0.1:8000/v1)

- it exposes the same endpoints (/v1/chat/completions, /v1/completions) as OpenAI

- accepts the same request schema (messages=[{"role": "..."}])

- the same client interface as OpenAI

- The OpenAI SDK doesn't know (or care) that it isn’t talking to OpenAI — it just sends HTTP requests to the specified url in the expected format

An **alternate** way is to send POST requests using python's **requests** module.


In [46]:
response = generate("compressed_model", "Explain quantum computing simply?")
print(response)

[2025-12-01 15:09:41] INFO _client.py:1025: HTTP Request: POST http://127.0.0.1:8000/v1/chat/completions "HTTP/1.1 200 OK"


Quantum computing refers to a branch of computing that operates using the principles of quantum mechanics. It is distinct from classical computing, which operates using bits (0 and 1) stored in memory on a computer's chip.

Under the theory of quantum mechanics, quantum bits (qubits) can exist in multiple states simultaneously, making them an ideal candidates for computation. Quantum computing uses qubits to perform a variety of operations, including arithmetic, logical operations, and quantum simulations.

Here's how quantum computing works:

1. Quantum bits are encoded into a state according to their computational value.

2. The state of the qubits is measured, and the results are processed using classical computers.

3. The results are then transformed back into the original state by applying classical operations such as rotation, flipping, and addition.

4. The process is repeated multiple times, until an answer is obtained.

The advantage of quantum computing is that it offers an 